In [1]:
# Load the libraries
import numpy as np 
import pandas as pd 
import seaborn as sns
import re
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from warnings import filterwarnings
filterwarnings('ignore')

from scipy.stats.mstats import winsorize
import scipy.stats as stats
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,explained_variance_score
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import (GridSearchCV, cross_val_score, cross_val_predict, StratifiedKFold, learning_curve)
from sklearn.metrics import confusion_matrix, classification_report

from statsmodels.tools.eval_measures import mse, rmse
from sklearn import preprocessing
## Display all the columns of the dataframe
pd.pandas.set_option('display.max_columns', None)

In [2]:
df=pd.read_csv("PRT_Q-filtered-2023-05-26.csv")
df.head(2)

,ref_area.label,indicator.label,source.label,sex.label,classif1.label,classif2.label,time,obs_value,obs_status.label,note_classif.label,note_indicator.label,note_source.label
0,Portugal,Employment by sex and occupation (thousands),LFS - EU Labour Force Survey,Sex: Total,Occupation (Skill level): Skill levels 3 and 4...,NaN,2022Q4,1944.0,NaN,NaN,Frequency: Quarterly,NaN
1,Portugal,Employment by sex and occupation (thousands),LFS - EU Labour Force Survey,Sex: Total,Occupation (Skill level): Skill level 2 ~ medium,NaN,2022Q4,2513.0,NaN,NaN,Frequency: Quarterly,NaN


In [3]:
df.tail(2)

,ref_area.label,indicator.label,source.label,sex.label,classif1.label,classif2.label,time,obs_value,obs_status.label,note_classif.label,note_indicator.label,note_source.label
1150,Portugal,Employment by sex and occupation (thousands),LFS - EU Labour Force Survey,Sex: Female,Occupation (ISCO-08): 4. Clerical support workers,NaN,2011Q1,269.0,Break in series,NaN,Frequency: Quarterly | Break in series: Method...,NaN
1151,Portugal,Employment by sex and occupation (thousands),LFS - EU Labour Force Survey,Sex: Female,Occupation (ISCO-08): 7. Craft and related tra...,NaN,2011Q1,126.0,Break in series,NaN,Frequency: Quarterly | Break in series: Method...,NaN


In [4]:
df.shape

(1152, 12)

In [5]:
df.isnull().sum()

ref_area.label           144
indicator.label          144
source.label             144
sex.label                144
classif1.label           144
classif2.label          1152
time                     144
obs_value                144
obs_status.label        1089
note_classif.label      1152
note_indicator.label     144
note_source.label       1152
dtype: int64

In [6]:
df.head(1)

,ref_area.label,indicator.label,source.label,sex.label,classif1.label,classif2.label,time,obs_value,obs_status.label,note_classif.label,note_indicator.label,note_source.label
0,Portugal,Employment by sex and occupation (thousands),LFS - EU Labour Force Survey,Sex: Total,Occupation (Skill level): Skill levels 3 and 4...,NaN,2022Q4,1944.0,NaN,NaN,Frequency: Quarterly,NaN


In [7]:
df= df.drop(["indicator.label","ref_area.label","source.label","classif2.label","obs_status.label","note_classif.label","note_indicator.label","note_source.label"], axis=1)

In [8]:
df=df[["sex.label", "classif1.label", "time", "obs_value"]]

In [9]:
df.head(2)

,sex.label,classif1.label,time,obs_value
0,Sex: Total,Occupation (Skill level): Skill levels 3 and 4...,2022Q4,1944.0
1,Sex: Total,Occupation (Skill level): Skill level 2 ~ medium,2022Q4,2513.0


In [10]:
label_counts = df['sex.label'].value_counts()
label_counts

sex.label
Sex: Total     336
Sex: Male      336
Sex: Female    336
Name: count, dtype: int64

In [11]:
df = df[df['sex.label'] != 'Sex: Total']

In [12]:
df["sex.label"].unique()

array(['Sex: Male', 'Sex: Female', nan], dtype=object)

In [13]:
df["classif1.label"].unique()

array(['Occupation (Skill level): Skill levels 3 and 4 ~ high',
       'Occupation (Skill level): Skill level 2 ~ medium',
       'Occupation (Skill level): Skill level 1 ~ low', nan,
       'Occupation (ISCO-08): 1. Managers',
       'Occupation (ISCO-08): 3. Technicians and associate professionals',
       'Occupation (ISCO-08): 4. Clerical support workers',
       'Occupation (ISCO-08): 7. Craft and related trades workers'],
      dtype=object)

In [14]:
df["time"].unique()

array(['2022Q4', nan, '2022Q3', '2022Q2', '2022Q1', '2021Q4', '2021Q3',
       '2021Q2', '2021Q1', '2020Q4', '2020Q3', '2020Q2', '2020Q1',
       '2019Q4', '2019Q3', '2019Q2', '2019Q1', '2018Q4', '2018Q3',
       '2018Q2', '2018Q1', '2017Q4', '2017Q3', '2017Q2', '2017Q1',
       '2016Q4', '2016Q3', '2016Q2', '2016Q1', '2015Q4', '2015Q3',
       '2015Q2', '2015Q1', '2014Q4', '2014Q3', '2014Q2', '2014Q1',
       '2013Q4', '2013Q3', '2013Q2', '2013Q1', '2012Q4', '2012Q3',
       '2012Q2', '2012Q1', '2011Q4', '2011Q3', '2011Q2', '2011Q1'],
      dtype=object)

In [15]:
#df['gender'] = df['sex.label'].str.extract('(Male|Female)')

In [16]:
#df['male'] = df['gender'].apply(lambda x: 1 if x == 'Male' else 0)
#df['female'] = df['gender'].apply(lambda x: 1 if x == 'Female' else 0)

In [17]:
#df.drop(columns=['sex.label'], inplace=True)

In [18]:
df.head(10)

,sex.label,classif1.label,time,obs_value
3,Sex: Male,Occupation (Skill level): Skill levels 3 and 4...,2022Q4,952.0
4,Sex: Male,Occupation (Skill level): Skill level 2 ~ medium,2022Q4,1361.0
5,Sex: Male,Occupation (Skill level): Skill level 1 ~ low,2022Q4,135.0
6,Sex: Female,Occupation (Skill level): Skill levels 3 and 4...,2022Q4,992.0
7,Sex: Female,Occupation (Skill level): Skill level 2 ~ medium,2022Q4,1152.0
8,Sex: Female,Occupation (Skill level): Skill level 1 ~ low,2022Q4,289.0
9,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN
15,Sex: Male,Occupation (ISCO-08): 1. Managers,2022Q4,184.0
16,Sex: Male,Occupation (ISCO-08): 3. Technicians and assoc...,2022Q4,301.0


In [19]:
df["classif1.label"].unique()

array(['Occupation (Skill level): Skill levels 3 and 4 ~ high',
       'Occupation (Skill level): Skill level 2 ~ medium',
       'Occupation (Skill level): Skill level 1 ~ low', nan,
       'Occupation (ISCO-08): 1. Managers',
       'Occupation (ISCO-08): 3. Technicians and associate professionals',
       'Occupation (ISCO-08): 4. Clerical support workers',
       'Occupation (ISCO-08): 7. Craft and related trades workers'],
      dtype=object)

In [20]:
df.shape

(816, 4)

In [21]:
df.isnull().sum()

sex.label         144
classif1.label    144
time              144
obs_value         144
dtype: int64

In [22]:
df.dropna(inplace=True)

In [23]:
df.isnull().sum()

sex.label         0
classif1.label    0
time              0
obs_value         0
dtype: int64

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 672 entries, 3 to 1151
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   sex.label       672 non-null    object 
 1   classif1.label  672 non-null    object 
 2   time            672 non-null    object 
 3   obs_value       672 non-null    float64
dtypes: float64(1), object(3)
memory usage: 26.2+ KB


In [25]:
df=df.rename(columns={'time':'Year_Quarter', 'obs_value':'Number_of_employees', 'Clerical support workers':'Clerical_support_workers',
                      'Skill levels 3 and 4 ~ high':'Skill_levels_high', 'Skill level 2 ~ medium':'Skill_level_medium',
                      'Skill level 1 ~ low':'Skill_level_low'})

In [26]:
df["classif1.label"].unique()

array(['Occupation (Skill level): Skill levels 3 and 4 ~ high',
       'Occupation (Skill level): Skill level 2 ~ medium',
       'Occupation (Skill level): Skill level 1 ~ low',
       'Occupation (ISCO-08): 1. Managers',
       'Occupation (ISCO-08): 3. Technicians and associate professionals',
       'Occupation (ISCO-08): 4. Clerical support workers',
       'Occupation (ISCO-08): 7. Craft and related trades workers'],
      dtype=object)

In [27]:
new_df = pd.DataFrame()

# create a column for each label, filled with zeros
new_df['Managers'] = 0
new_df['Technicians'] = 0
new_df['Clerical_workers'] = 0
new_df['trades_workers'] = 0
new_df['Skill levels 3 and 4 ~ high'] = 0
new_df['Skill level 2 ~ medium'] = 0
new_df['Skill level 1 ~ low'] = 0


# loop through each row of the original dataframe
for index, row in df.iterrows():

    # get the label from the current row
    label = row['classif1.label']

    # set the value in the appropriate new column to 1
    if label == 'Occupation (ISCO-08): 4. Clerical support workers':
        new_df.at[index, 'Clerical_workers'] = 1
    elif label == 'Occupation (ISCO-08): 1. Managers':
        new_df.at[index, 'Managers'] = 1
    elif label == 'Occupation (ISCO-08): 3. Technicians and associate professionals':
        new_df.at[index, 'Technicians'] = 1
    elif label == 'Occupation (ISCO-08): 7. Craft and related trades workers':
        new_df.at[index, 'trades_workers'] = 1
    elif label == 'Occupation (Skill level): Skill levels 3 and 4 ~ high':
        new_df.at[index, 'Skill levels 3 and 4 ~ high'] = 1
    elif label == 'Occupation (Skill level): Skill level 2 ~ medium':
        new_df.at[index, 'Skill level 2 ~ medium'] = 1
    elif label == 'Occupation (Skill level): Skill level 1 ~ low':
        new_df.at[index, 'Skill level 1 ~ low'] = 1

# concatenate the original dataframe and the new dataframe
result_df = pd.concat([df, new_df], axis=1)

# print the resulting dataframe
result_df.head()


,sex.label,classif1.label,Year_Quarter,Number_of_employees,Managers,Technicians,Clerical_workers,trades_workers,Skill levels 3 and 4 ~ high,Skill level 2 ~ medium,Skill level 1 ~ low
3,Sex: Male,Occupation (Skill level): Skill levels 3 and 4...,2022Q4,952.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN
4,Sex: Male,Occupation (Skill level): Skill level 2 ~ medium,2022Q4,1361.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
5,Sex: Male,Occupation (Skill level): Skill level 1 ~ low,2022Q4,135.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
6,Sex: Female,Occupation (Skill level): Skill levels 3 and 4...,2022Q4,992.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN
7,Sex: Female,Occupation (Skill level): Skill level 2 ~ medium,2022Q4,1152.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [28]:
# drop the original 'classif1.label' column
result_df = result_df.drop(['classif1.label'], axis=1)

In [29]:
result_df.isnull().sum()

sex.label                        0
Year_Quarter                     0
Number_of_employees              0
Managers                       576
Technicians                    576
Clerical_workers               576
trades_workers                 576
Skill levels 3 and 4 ~ high    576
Skill level 2 ~ medium         576
Skill level 1 ~ low            576
dtype: int64

In [30]:
result_df.shape

(672, 10)

In [31]:
result_df.fillna(0, inplace=True)

In [32]:
#result_df=result_df.drop(["Construction","gender"],axis=1)

In [33]:
result_df.tail()

,sex.label,Year_Quarter,Number_of_employees,Managers,Technicians,Clerical_workers,trades_workers,Skill levels 3 and 4 ~ high,Skill level 2 ~ medium,Skill level 1 ~ low
1146,Sex: Male,2011Q1,630.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1148,Sex: Female,2011Q1,101.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1149,Sex: Female,2011Q1,154.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1150,Sex: Female,2011Q1,269.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1151,Sex: Female,2011Q1,126.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [34]:
result_df.head()

,sex.label,Year_Quarter,Number_of_employees,Managers,Technicians,Clerical_workers,trades_workers,Skill levels 3 and 4 ~ high,Skill level 2 ~ medium,Skill level 1 ~ low
3,Sex: Male,2022Q4,952.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,Sex: Male,2022Q4,1361.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,Sex: Male,2022Q4,135.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,Sex: Female,2022Q4,992.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,Sex: Female,2022Q4,1152.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [35]:
df=result_df.rename(columns={'time':'Year_Quarter', 'obs_value':'Number_of_employees','Skill levels 3 and 4 ~ high':'Skill_levels_high', 'Skill level 2 ~ medium':'Skill_level_medium',
                      'Skill level 1 ~ low':'Skill_level_low'})

In [36]:
df.tail()

,sex.label,Year_Quarter,Number_of_employees,Managers,Technicians,Clerical_workers,trades_workers,Skill_levels_high,Skill_level_medium,Skill_level_low
1146,Sex: Male,2011Q1,630.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1148,Sex: Female,2011Q1,101.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1149,Sex: Female,2011Q1,154.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1150,Sex: Female,2011Q1,269.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1151,Sex: Female,2011Q1,126.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 672 entries, 3 to 1151
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   sex.label            672 non-null    object 
 1   Year_Quarter         672 non-null    object 
 2   Number_of_employees  672 non-null    float64
 3   Managers             672 non-null    float64
 4   Technicians          672 non-null    float64
 5   Clerical_workers     672 non-null    float64
 6   trades_workers       672 non-null    float64
 7   Skill_levels_high    672 non-null    float64
 8   Skill_level_medium   672 non-null    float64
 9   Skill_level_low      672 non-null    float64
dtypes: float64(8), object(2)
memory usage: 57.8+ KB


In [38]:
df["Number_of_employees"] = df["Number_of_employees"].astype('int')
df["Managers"] = df["Managers"].astype('int')
df["Technicians"] = df["Technicians"].astype('int')
df["Clerical_workers"] = df["Clerical_workers"].astype('int')
df["Skill_levels_high"] = df["Skill_levels_high"].astype('int')
df["Skill_level_medium"] = df["Skill_level_medium"].astype('int')
df["Skill_level_low"] = df["Skill_level_low"].astype('int')

In [39]:
df['gender'] = df['sex.label'].str.extract('(Male|Female)')

In [40]:
df['male'] = df['gender'].apply(lambda x: 1 if x == 'Male' else 0)
df['female'] = df['gender'].apply(lambda x: 1 if x == 'Female' else 0)

In [41]:
df.drop(columns=['sex.label'], inplace=True)

In [42]:
df.drop(columns=['gender'], inplace=True)

In [43]:
df.tail()

,Year_Quarter,Number_of_employees,Managers,Technicians,Clerical_workers,trades_workers,Skill_levels_high,Skill_level_medium,Skill_level_low,male,female
1146,2011Q1,630,0,0,0,1.0,0,0,0,1,0
1148,2011Q1,101,1,0,0,0.0,0,0,0,0,1
1149,2011Q1,154,0,1,0,0.0,0,0,0,0,1
1150,2011Q1,269,0,0,1,0.0,0,0,0,0,1
1151,2011Q1,126,0,0,0,1.0,0,0,0,0,1


In [44]:
df=df[['Year_Quarter','male','female','Number_of_employees','Managers','Technicians','Clerical_workers','trades_workers','Skill_levels_high','Skill_level_medium','Skill_level_low']]

In [45]:
df.shape

(672, 11)

In [108]:
#portugal_ml = df.to_csv('portugal_ml.csv', index=False)

## Statistics for Data Analytics Tasks

## 1. T-Test One Population - Number_of_employees in Construction industry in Portugal.

In [47]:
df["Number_of_employees"].mean()

463.53125

In [48]:
X= df["Number_of_employees"];X

3        952
4       1361
5        135
6        992
7       1152
        ... 
1146     630
1148     101
1149     154
1150     269
1151     126
Name: Number_of_employees, Length: 672, dtype: int32

In [49]:
#H0 : u = 463
#H1 : u =! 463
stats.ttest_1samp(X,463)

Ttest_1sampResult(statistic=0.033658720245011, pvalue=0.9731593064482836)

In [50]:
##  statistic=0.016522005465077535 is the t value = 0.01

In [51]:
##  if p-value < 0.05  we reject Ho
## if p-value > 0.05 we accept tht Ho 
## the p- vlaue is 0.973 is greater than 0.05 so we accept Ho

In [52]:
## We accept H0 ---> so there is enough evidence to say that the average Number_of_employees 463,Quarterly.

## 2. T-Test - two populations Qarterly Number of employees in Q1 and Q4 

In [53]:
df1=df[['Year_Quarter','Number_of_employees']]

In [54]:
df1.tail()

,Year_Quarter,Number_of_employees
1146,2011Q1,630
1148,2011Q1,101
1149,2011Q1,154
1150,2011Q1,269
1151,2011Q1,126


In [55]:
#We group the data according to the categories requested.
Q1 = df1.query('Year_Quarter == "2020Q1"')['Number_of_employees']
Q4 = df1.query('Year_Quarter == "2020Q4"')['Number_of_employees']

In [56]:
#We perform the test assuming as H0 that mu1 = mu2 (Q1 Number_of_employees = Q4 Number_of_employees)
test = stats.ttest_ind(Q1, Q4, equal_var=True)
display(test)

Ttest_indResult(statistic=-0.012818851477316996, pvalue=0.9898701749442022)

In [57]:
## pvalue=0.98 >alpha(0.05) means we accept the H0, there is no significant difference in Q1 Number_of_employees and Q4 Number_of_employees.

In [58]:
#We group the data according to the categories requested.
Q1 = df1.query('Year_Quarter == "2011Q1"')['Number_of_employees']
Q4 = df1.query('Year_Quarter == "2011Q4"')['Number_of_employees']

In [59]:
test = stats.ttest_ind(Q1, Q4, equal_var=True)
display(test)

Ttest_indResult(statistic=0.09818368571066366, pvalue=0.9225394690910096)

In [60]:
 ## p-value 0.92 is > alpha 0.05 so we accept the H0,

## 3. Confidence Intervel 

In [61]:
import scipy.stats as st

In [62]:
df.head()

,Year_Quarter,male,female,Number_of_employees,Managers,Technicians,Clerical_workers,trades_workers,Skill_levels_high,Skill_level_medium,Skill_level_low
3,2022Q4,1,0,952,0,0,0,0.0,1,0,0
4,2022Q4,1,0,1361,0,0,0,0.0,0,1,0
5,2022Q4,1,0,135,0,0,0,0.0,0,0,1
6,2022Q4,0,1,992,0,0,0,0.0,1,0,0
7,2022Q4,0,1,1152,0,0,0,0.0,0,1,0


In [63]:
df["Number_of_employees"].describe()

count     672.000000
mean      463.531250
std       409.153226
min        73.000000
25%       149.000000
50%       261.500000
75%       758.000000
max      1619.000000
Name: Number_of_employees, dtype: float64

In [64]:
#DEFINE THE VARIABLE WE FIND THE CONFIDENCE INTERVEL OF Number_of_employees.
X = df.iloc[:, 2:3].values
X

array([[0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
    

In [65]:
#CENTRAL TENDENCY FOR THE Number_of_employees
mean = X.mean()
print("The mean is ", mean)

The mean is  0.5


In [66]:
#create 90% confidence interval for population mean of Number_of_employees per Quater
st.t.interval(alpha=0.95, df=len(X)-1, loc=np.mean(X), scale=st.sem(X)) 

(array([0.46209985]), array([0.53790015]))

In [67]:
## The population average Number_of_employees is between 432 and 494.

## Chi Squared test

In [68]:
from numpy import sqrt, abs, round
import scipy.stats as stats
from scipy.stats import norm

In [69]:
df.head()

,Year_Quarter,male,female,Number_of_employees,Managers,Technicians,Clerical_workers,trades_workers,Skill_levels_high,Skill_level_medium,Skill_level_low
3,2022Q4,1,0,952,0,0,0,0.0,1,0,0
4,2022Q4,1,0,1361,0,0,0,0.0,0,1,0
5,2022Q4,1,0,135,0,0,0,0.0,0,0,1
6,2022Q4,0,1,992,0,0,0,0.0,1,0,0
7,2022Q4,0,1,1152,0,0,0,0.0,0,1,0


In [70]:
df["female"].unique()

array([0, 1], dtype=int64)

In [71]:
df["Clerical_workers"].unique()

array([0, 1])

In [72]:
## Null Hypothesis: There is significant association between female and Clerical_support_workers features.

## Alternate Hypothesis: There is no significant association between female and Clerical_support_workers features.

In [73]:
dataset_table=pd.crosstab(df['female'],df['Clerical_workers'])
dataset_table

Clerical_workers,0,1
female,,
0,288,48
1,288,48


In [74]:
observed=dataset_table.values
val2=stats.chi2_contingency(dataset_table)
expected=val2[3]

In [75]:
alpha=0.5
ddof =1

In [76]:
from scipy.stats import chi2
chi_square=sum([(o-e)**2./e for o,e in zip(observed,expected)])
chi_square_statistic=chi_square[0]+chi_square[1]
p_value=1-chi2.cdf(x=chi_square_statistic,df=1)
     

In [77]:
print("chi-square statistic:-",chi_square_statistic)

print('Significance level: ',alpha)

print('Degree of Freedom: ',ddof)

print('p-value:',p_value)  

chi-square statistic:- 0.0
Significance level:  0.5
Degree of Freedom:  1
p-value: 1.0


In [78]:
if p_value<=alpha:
    print("Reject Null Hypothesis")
else:
    print("Accept Null Hypthesis")

Accept Null Hypthesis


In [79]:
## Since the p-value is greater than 0.05, we accept the null hypothesis. We conclude that There is significant association between female and Clerical_support_workers features.

## Non-Parametric Tests ## Need to split the notebook from here.

## 1. Sign Test

In [80]:
## Apply the sign test to compare the median Number_of_employees per Year_Quarter between the years 2011Q1 and 2020Q4.

In [81]:
## H0 : The median Number_of_employees per Year_Quarter in construction industry are the same for the years between 2011Q1 and 2020Q4.
## H1 : The median Number_of_employees per Year_Quarter in construction industry different for the years between 2011Q1 and 2020Q4.

In [82]:
df.tail()

,Year_Quarter,male,female,Number_of_employees,Managers,Technicians,Clerical_workers,trades_workers,Skill_levels_high,Skill_level_medium,Skill_level_low
1146,2011Q1,1,0,630,0,0,0,1.0,0,0,0
1148,2011Q1,0,1,101,1,0,0,0.0,0,0,0
1149,2011Q1,0,1,154,0,1,0,0.0,0,0,0
1150,2011Q1,0,1,269,0,0,1,0.0,0,0,0
1151,2011Q1,0,1,126,0,0,0,1.0,0,0,0


In [83]:
# subset the data for years 2011Q1 and 2020Q4
df_subset = df[df["Year_Quarter"].isin(["2011Q1", "2020Q4"])]
df_subset

,Year_Quarter,male,female,Number_of_employees,Managers,Technicians,Clerical_workers,trades_workers,Skill_levels_high,Skill_level_medium,Skill_level_low
195,2020Q4,1,0,980,0,0,0,0.0,1,0,0
196,2020Q4,1,0,1364,0,0,0,0.0,0,1,0
197,2020Q4,1,0,98,0,0,0,0.0,0,0,1
198,2020Q4,0,1,1010,0,0,0,0.0,1,0,0
199,2020Q4,0,1,1119,0,0,0,0.0,0,1,0
200,2020Q4,0,1,269,0,0,0,0.0,0,0,1
207,2020Q4,1,0,192,1,0,0,0.0,0,0,0
208,2020Q4,1,0,327,0,1,0,0.0,0,0,0
209,2020Q4,1,0,160,0,0,1,0.0,0,0,0
210,2020Q4,1,0,505,0,0,0,1.0,0,0,0


In [84]:
# calculate the difference in Number_of_employees between the years 2011Q1 and 2020Q4
diff = df_subset.loc[df_subset["Year_Quarter"] == "2011Q1","Number_of_employees"].iloc[0] - \
       df_subset.loc[df_subset["Year_Quarter"] == "2020Q4","Number_of_employees"].iloc[0]
diff

-263

In [85]:
# create a new column to indicate whether the difference is positive, negative, or zero
df_subset["diff_sign"] = df_subset["Number_of_employees"].apply(lambda x: "+" if x > diff else "-" if x < diff else "0")

In [86]:
df_subset["diff_sign"]

195     +
196     +
197     +
198     +
199     +
200     +
207     +
208     +
209     +
210     +
212     +
213     +
214     +
215     +
1131    +
1132    +
1133    +
1134    +
1135    +
1136    +
1143    +
1144    +
1145    +
1146    +
1148    +
1149    +
1150    +
1151    +
Name: diff_sign, dtype: object

In [87]:
# count the number of positive and negative differences
counts = df_subset["diff_sign"].value_counts()
counts

diff_sign
+    28
Name: count, dtype: int64

In [88]:
from scipy import stats

In [89]:
# calculate the p-value
p_value = stats.binom_test(counts, n=counts.sum(), p=0.5)

In [90]:
print(f"Test statistic: {counts}")
print(f"P-value: {p_value}")


Test statistic: diff_sign
+    28
Name: count, dtype: int64
P-value: 7.450580596923828e-09


In [91]:
## Accroding to the pvalue 0.0025 < alpha(0.05) we reject the H0, 
## means there is no evidence to say that The median Number_of_employees per Year_Quarter in construction industry are the same for the years between 2011Q1 and 2020Q4.

## 2. Kruskal-Wallis H test

In [92]:
from scipy.stats import kruskal

In [93]:
## H0: there is no difference in weekly earnings in Quater1 to Q4
## H1: there is at least one of the Quarterly earnings has differences 

In [94]:
df.tail()

,Year_Quarter,male,female,Number_of_employees,Managers,Technicians,Clerical_workers,trades_workers,Skill_levels_high,Skill_level_medium,Skill_level_low
1146,2011Q1,1,0,630,0,0,0,1.0,0,0,0
1148,2011Q1,0,1,101,1,0,0,0.0,0,0,0
1149,2011Q1,0,1,154,0,1,0,0.0,0,0,0
1150,2011Q1,0,1,269,0,0,1,0.0,0,0,0
1151,2011Q1,0,1,126,0,0,0,1.0,0,0,0


In [95]:
df_subset1 = df[["Year_Quarter","Number_of_employees"]]
df_subset1.head()

,Year_Quarter,Number_of_employees
3,2022Q4,952
4,2022Q4,1361
5,2022Q4,135
6,2022Q4,992
7,2022Q4,1152


In [96]:
##df.to_csv("df_subset1", index=False)

In [97]:
# group the data by quarter
q1 = df.loc[df['Year_Quarter'] == '2015Q1']['Number_of_employees']
q2 = df.loc[df['Year_Quarter'] == '2015Q2']['Number_of_employees']
q3 = df.loc[df['Year_Quarter'] == '2015Q3']['Number_of_employees']
q4 = df.loc[df['Year_Quarter'] == '2015Q4']['Number_of_employees']


In [98]:
# perform the Kruskal-Wallis H test
stat, p_value = kruskal(q1, q2, q3, q4)

In [99]:
# display the results
print('Kruskal-Wallis H Test Results:')
print('Test Statistic: {:.4f}'.format(stat))
print('p-value: {:.4f}'.format(p_value))

Kruskal-Wallis H Test Results:
Test Statistic: 0.0133
p-value: 0.9996


In [100]:
if p_value < 0.05:
    print('There is a significant difference between the quarters.')
else:
    print('There is no significant difference between the quarters.')

There is no significant difference between the quarters.


In [101]:
## According to the results Pvalue 0.98 is > alpha(0.05) so we accept the H0,
## means There is no significant difference between the Skilled_operatives weely earnings in quarters 

## 3. U-Mann Whitman test

In [102]:
df2=df[['Skill_levels_high','Skill_level_low']]

In [103]:
df2.head()

,Skill_levels_high,Skill_level_low
3,1,0
4,0,0
5,0,1
6,1,0
7,0,0


In [104]:
## H0: There is no difference Number of Skill_levels_high employees and Number of Skill_levels_low 
## H1: There is no difference Number of Skill_levels_high employees and Number of Skill_levels_low 

In [105]:
from scipy.stats import mannwhitneyu

In [106]:
mannwhitneyu(df2['Skill_levels_high'], df2['Skill_level_low'])

MannwhitneyuResult(statistic=225792.0, pvalue=1.0)

In [107]:
## according to the results pvalue 1.0 is > alpha(0.05) so we accept the H0,
## means there is no evidence to say that There is no difference Number of Skill_levels_high employees and Number of Skill_levels_low 